### Investigate final_score_unweighted zero values

In [1]:
%%capture

%run '../lib/libraries.ipynb'

dython              0.6.1


You should consider upgrading via the '/Users/duane.hinkley/PycharmProjects/c772-capstone-project/venv/bin/python -m pip install --upgrade pip' command.


tinydb              4.2.0


You should consider upgrading via the '/Users/duane.hinkley/PycharmProjects/c772-capstone-project/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
filterDf = load_df('filterDf')
types = get_var_types()

#### Create zero value dataframe

In [3]:
zeroDf = filterDf.filter(F.col('final_score_unweighted') == 0)

#### Count zero values

In [4]:
zeroDf.count()

16582

- 2,867 zero values

In [5]:
distinct_val(zeroDf)

,0
is_manual_scoring_required,False
final_score_unweighted,0


- No additional values

###### Count observations with no response

In [6]:
zeroDf.filter(F.col('assessment_item_response_id').isNull()).count()

13715

- 13,715

###### Exclude observations with no response

In [7]:
noResponseDf = zeroDf.filter(~F.col('assessment_item_response_id').isNull())

###### Remaining count

In [8]:
noResponseDf.count()

2867

In [9]:
- 2867

-2867

###### Are the remaining zero scores valid

In [10]:
pdDf = add_zero_raw_score_var(noResponseDf).toPandas()

crosstab_percent( pd.crosstab(pdDf.zero_score, pdDf.response_correctness) )

response_correctness,[unassigned],correct,incorrect,partially_correct
zero_score,,,,
No,0.0,96.0,0.0,4.0
Yes,1.0,0.0,99.0,0.0


- Remaining observations
  - raw_score = 0 (zero_score = Yes)
    - response_correctnss
      - 1% [unassigned]
      - 99% incorrect
  - raw_score > 0 (zero_score = No)
    - response_correctnss
      - 96% incorrect
      - 4% partially_correct

##### Conclusion:
- No further action needed
- The 2,867 zero values correct

In [11]:
finish_todo('Investigate final_score_unweighted zero values')